In [10]:
!pip install openai
!pip install pandas
!pip install plotly
!pip install python-dotenv



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [37]:
import pandas as pd

experience = pd.read_csv('/Users/alexripperton/PycharmProjects/YouTube/Qualitative Data Analysis/Demo qualitative analysis - Sheet1.csv')

experience_df = pd.DataFrame(experience)
experience_df

,experience_id,title,Location,summary,tenure
0,1,Head of Product Development at JP Media Group,"Florio, Brazil",Led product development from opportunity analy...,April 2009 - July 2017 (8 years 4 months)
1,2,Product Analyst at KP,"Paris, France",Developing satellite imagery analysis products...,March 2024 - Present (1 month)
2,3,Senior Developer at Smart Factory MTV Cribs,"Monterrey, CA",Improved development work of other developers ...,January 2018 - January 2020
3,4,Founder at Gmail,UK,"Established and managed business, specializing...",4+ years
4,5,Founder at BBK,Not specified,Not specified,Not specified
5,6,"Co Founder, Board Member at kp Bikes sp. z o.o.","Masovian District, Poland",Co-founded the company and managed supply chai...,January 2014 - February 2018 (4 years 2 months)


In [38]:
from openai import OpenAI
import os
import dotenv

dotenv.load_dotenv(
    dotenv.find_dotenv()
)
client = OpenAI(api_key= os.getenv("OPENAI_API_KEY"))

def classify_text(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",  # Confirm that you're using the correct model identifier
            messages=[
    {"role": "system", "content": "You are a classification agent."},
    {"role": "user", "content": f"Classify the following job description into Industry and Years of Experience. Your options for industry are the following (Tech, Marketing, Finance). Your output must be in the following format with no other characters: Industry, Years of Experience. Years of experience must only be an integer with no other characters. Example Input: Senior Data Analyst. Responsible for data analysis and reporting Tenure: Jan 2021 - March 2022. Example Output: Tech, 1.5 . Here is your Input: {text}"}
  ],
            max_tokens=1000
        )
        return str(response.choices[0].message.content)
    except Exception as e:
        print(f"API call failed: {e}")
        return "Unknown, Unknown"  # Return a default value in case of API failure



In [39]:
def extract_info(row):
    combined_text = f"{row['title']} {row['summary']} {row['tenure']}"
    classification = classify_text(combined_text)
    parts = classification.split(', ')
    if len(parts) != 2:
        return pd.Series(['Unknown'] * 2)  # Ensure two elements are returned
    return pd.Series(parts)


In [43]:
# Apply the function and ensure the DataFrame is properly updated

experience_df[['Industry', 'Years Experience']] = experience_df.apply(extract_info, axis=1)


experience_df.to_csv('grouped_experience_rd2.csv')

experience_df


,experience_id,title,Location,summary,tenure,Industry,Years Experience
0,1,Head of Product Development at JP Media Group,"Florio, Brazil",Led product development from opportunity analy...,April 2009 - July 2017 (8 years 4 months),Marketing,8
1,2,Product Analyst at KP,"Paris, France",Developing satellite imagery analysis products...,March 2024 - Present (1 month),Finance,0
2,3,Senior Developer at Smart Factory MTV Cribs,"Monterrey, CA",Improved development work of other developers ...,January 2018 - January 2020,Tech,2
3,4,Founder at Gmail,UK,"Established and managed business, specializing...",4+ years,Tech,4
4,5,Founder at BBK,Not specified,Not specified,Not specified,Marketing,0
5,6,"Co Founder, Board Member at kp Bikes sp. z o.o.","Masovian District, Poland",Co-founded the company and managed supply chai...,January 2014 - February 2018 (4 years 2 months),Tech,4


In [47]:
import plotly.express as px


industry_counts = experience_df['Industry'].value_counts().head(5)


# Step 3: Create a bar chart using Plotly
fig = px.bar(industry_counts, title="Most Common Job Industries")
fig.show()

In [46]:
import plotly.express as px


industry_counts = experience_df['Years Experience'].value_counts().head(5)


# Step 3: Create a bar chart using Plotly
fig = px.bar(industry_counts, title="Most Common Job Tenures")
fig.show()